In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [2]:
x = pd.read_csv("./data/processed/csp_bond_types.csv", index_col=0)
y = pd.read_csv("./data/processed/energy.csv")

non_nan_rows = (x.sum(axis=1)>0).to_list()

x = x.loc[non_nan_rows].to_numpy()
y = y.loc[non_nan_rows].to_numpy()

x.shape, y.shape

((138359, 26), (138359, 1))

In [3]:
from sklearn.model_selection import train_test_split

# Split the data
x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size=.2, shuffle=True, random_state=42)

In [4]:
x_train.shape, y_train.shape, x_valid.shape, y_valid.shape

((110687, 26), (110687, 1), (27672, 26), (27672, 1))

## Trianing the model

In [5]:
import os

work_dir = 'mansur/experiment_10'

os.mkdir(f'{work_dir}')
os.mkdir(f'{work_dir}/checkpoint')
os.mkdir(f'{work_dir}/model')

In [6]:
from tensorflow.keras.layers import Dense as d
from tensorflow.keras.layers import InputLayer as i

# idea for future
# def neg_relu(x):
#     return tf.nn.relu(x) * -1

model = tf.keras.Sequential([
    i(x.shape[1]),
    d(100),
])

model.compile(
    optimizer="Adam",
    loss="MSE",
    metrics="MAE"
)

Metal device set to: Apple M1


2022-06-25 02:12:29.820342: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-06-25 02:12:29.820836: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [7]:
model.build()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               2700      
                                                                 
Total params: 2,700
Trainable params: 2,700
Non-trainable params: 0
_________________________________________________________________


In [8]:
from tensorflow.keras.callbacks import CSVLogger, ModelCheckpoint

csv_logger = CSVLogger(f'{work_dir}/training.log', append=True)

model_checkpoint_callback = ModelCheckpoint(
    filepath=f'{work_dir}/checkpoint/weights',
    save_weights_only=True,
    monitor='val_MAE',
    mode='min',
    save_best_only=True
)

model.fit(
    x_train, y_train,
    epochs = 500,
    initial_epoch=0,
    validation_split=.2,
    callbacks=[csv_logger, model_checkpoint_callback],
    verbose=True
)

model.save(f'{work_dir}/model')

Epoch 1/500


2022-06-25 02:18:48.809090: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-06-25 02:18:48.997017: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


2768/2768 [==============================] - ETA: 0s - loss: 1553161.0000 - MAE: 1188.0322

2022-06-25 02:18:59.955325: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


2768/2768 [==============================] - 13s 4ms/step - loss: 1553161.0000 - MAE: 1188.0322 - val_loss: 1430784.5000 - val_MAE: 1132.3802
Epoch 2/500
2768/2768 [==============================] - 12s 4ms/step - loss: 1296599.3750 - MAE: 1072.7427 - val_loss: 1189288.5000 - val_MAE: 1018.0193
Epoch 3/500
2768/2768 [==============================] - 12s 4ms/step - loss: 1069623.5000 - MAE: 958.8203 - val_loss: 975879.8125 - val_MAE: 904.4897
Epoch 4/500
2768/2768 [==============================] - 12s 4ms/step - loss: 870371.8125 - MAE: 845.6370 - val_loss: 789938.9375 - val_MAE: 791.7698
Epoch 5/500
2768/2768 [==============================] - 13s 5ms/step - loss: 698387.5625 - MAE: 733.3400 - val_loss: 631187.9375 - val_MAE: 680.0721
Epoch 6/500
2768/2768 [==============================] - 13s 5ms/step - loss: 553318.2500 - MAE: 622.2778 - val_loss: 499067.2500 - val_MAE: 569.5367
Epoch 7/500
2768/2768 [==============================] - 12s 4ms/step - loss: 434998.5312 - MAE: 512.78

...

2768/2768 [==============================] - 12s 4ms/step - loss: 67929.4297 - MAE: 63.3673 - val_loss: 72135.6094 - val_MAE: 65.8923
Epoch 330/500
2768/2768 [==============================] - 12s 4ms/step - loss: 67817.9219 - MAE: 63.3824 - val_loss: 72017.7734 - val_MAE: 65.8379
Epoch 331/500
2768/2768 [==============================] - 12s 4ms/step - loss: 67707.9531 - MAE: 63.4812 - val_loss: 71902.4609 - val_MAE: 65.5781
Epoch 332/500
2768/2768 [==============================] - 12s 4ms/step - loss: 67595.0156 - MAE: 63.0903 - val_loss: 71778.9062 - val_MAE: 66.2770
Epoch 333/500
2768/2768 [==============================] - 12s 4ms/step - loss: 67481.8984 - MAE: 63.6143 - val_loss: 71663.3125 - val_MAE: 65.1211
Epoch 334/500
2768/2768 [==============================] - 12s 4ms/step - loss: 67370.8281 - MAE: 63.1202 - val_loss: 71542.4141 - val_MAE: 65.4217
Epoch 335/500
2768/2768 [==============================] - 12s 4ms/step - loss: 67258.3438 - MAE: 63.3626 - val_loss: 71426.67

2768/2768 [==============================] - 12s 4ms/step - loss: 61923.6445 - MAE: 61.2461 - val_loss: 65750.4688 - val_MAE: 63.3266
Epoch 385/500
2768/2768 [==============================] - 12s 4ms/step - loss: 61816.9453 - MAE: 61.2723 - val_loss: 65637.9531 - val_MAE: 63.1875
Epoch 386/500
2768/2768 [==============================] - 12s 4ms/step - loss: 61709.9883 - MAE: 61.0901 - val_loss: 65522.0000 - val_MAE: 63.5177
Epoch 387/500
2768/2768 [==============================] - 12s 4ms/step - loss: 61604.4062 - MAE: 61.2938 - val_loss: 65412.9727 - val_MAE: 63.0295
Epoch 388/500
2768/2768 [==============================] - 12s 4ms/step - loss: 61496.6992 - MAE: 60.8743 - val_loss: 65295.9180 - val_MAE: 63.7654
Epoch 389/500
2768/2768 [==============================] - 12s 4ms/step - loss: 61391.9844 - MAE: 61.1881 - val_loss: 65184.9883 - val_MAE: 63.4171
Epoch 390/500
2768/2768 [==============================] - 12s 4ms/step - loss: 61287.1641 - MAE: 61.1312 - val_loss: 65074.24

2768/2768 [==============================] - 12s 4ms/step - loss: 56229.4922 - MAE: 59.1108 - val_loss: 59694.1992 - val_MAE: 61.6223
Epoch 440/500
2768/2768 [==============================] - 12s 4ms/step - loss: 56128.8203 - MAE: 59.3655 - val_loss: 59589.6562 - val_MAE: 61.0903
Epoch 441/500
2768/2768 [==============================] - 12s 4ms/step - loss: 56029.4766 - MAE: 59.1288 - val_loss: 59482.9805 - val_MAE: 61.1702
Epoch 442/500
2768/2768 [==============================] - 12s 4ms/step - loss: 55929.1758 - MAE: 59.2346 - val_loss: 59377.7773 - val_MAE: 60.8821
Epoch 443/500
2768/2768 [==============================] - 12s 4ms/step - loss: 55828.1367 - MAE: 59.1692 - val_loss: 59269.4961 - val_MAE: 60.8876
Epoch 444/500
2768/2768 [==============================] - 12s 4ms/step - loss: 55727.7148 - MAE: 59.1238 - val_loss: 59165.3672 - val_MAE: 60.6717
Epoch 445/500
2768/2768 [==============================] - 12s 4ms/step - loss: 55627.5391 - MAE: 58.9076 - val_loss: 59054.96

2768/2768 [==============================] - 12s 4ms/step - loss: 50828.1992 - MAE: 57.4675 - val_loss: 53956.2969 - val_MAE: 58.6706
Epoch 495/500
2768/2768 [==============================] - 12s 4ms/step - loss: 50733.0312 - MAE: 57.1186 - val_loss: 53849.6484 - val_MAE: 59.1546
Epoch 496/500
2768/2768 [==============================] - 12s 4ms/step - loss: 50637.1289 - MAE: 57.2895 - val_loss: 53748.9766 - val_MAE: 59.0338
Epoch 497/500
2768/2768 [==============================] - 12s 4ms/step - loss: 50542.1992 - MAE: 57.2429 - val_loss: 53648.6641 - val_MAE: 58.9583
Epoch 498/500
2768/2768 [==============================] - 12s 4ms/step - loss: 50447.7227 - MAE: 57.1844 - val_loss: 53547.6211 - val_MAE: 58.9040
Epoch 499/500
2768/2768 [==============================] - 12s 4ms/step - loss: 50353.5781 - MAE: 57.1214 - val_loss: 53447.5547 - val_MAE: 58.9778
Epoch 500/500
2768/2768 [==============================] - 12s 4ms/step - loss: 50256.7188 - MAE: 56.9485 - val_loss: 53343.89

2022-06-25 03:59:44.054636: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


In [9]:
model.evaluate(x_valid, y_valid)

865/865 [==============================] - 3s 3ms/step - loss: 49256.3008 - MAE: 56.8920


[49256.30078125, 56.89198684692383]